In [119]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


## Manhattan Data

Use the data and transformation code for NY neighborhoods from tutorial notebook

In [9]:
import urllib.request
url = "https://cocl.us/new_york_dataset"
filename = 'newyork_data.json'
urllib.request.urlretrieve(url, filename)


print('Data downloaded!')

Data downloaded!


In [11]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)
neighborhoods_data = newyork_data['features']
neighborhoods_data[0]

# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

Again this analysis focuses on Manhattan only, therefore drop all other boroughs

In [139]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


## Manhattan Population Data

For further analysis population is needed, as population density and total population in each neighborhood is an important factor influencing health care and demand for pharmaceutical products. We can find some good public data with population for each neighborhood in NYC at https://data.cityofnewyork.us.

In [128]:
#import manhattan population data
link="https://data.cityofnewyork.us/api/views/8m6s-esnp/rows.csv?accessType=DOWNLOAD"
manhattan_pop= pd.read_csv(link)
manhattan_pop.head()

,Borough,Year,FIPS County Code,NTA Code,NTA Name,Population
0,Manhattan,2010,61,MN01,Marble Hill-Inwood,46746
1,Manhattan,2010,61,MN03,Central Harlem North-Polo Grounds,75282
2,Manhattan,2010,61,MN04,Hamilton Heights,48520
3,Manhattan,2010,61,MN06,Manhattanville,22950
4,Manhattan,2010,61,MN09,Morningside Heights,55929


In [129]:
#ommit all other features which are not useful for further analysis
manhattan_pop= manhattan_pop[["NTA Name","Population"]]
manhattan_pop.head()

,NTA Name,Population
0,Marble Hill-Inwood,46746
1,Central Harlem North-Polo Grounds,75282
2,Hamilton Heights,48520
3,Manhattanville,22950
4,Morningside Heights,55929


For the imported population dataset some neighborhoods are summarized within one row. In order to have population data for all neighborhoods we do the following:
1) transform the NTA Names to a list containing the certain neighborhoods
2) add length of each neighborhood-list as new column
3) iterate over dataframe and append all neighborhoods as single to a new list. For Neigborhoods that have been summarized we take the mean (which is the population divided by number of neighborhoods)

In [130]:
#transform the NTA Names to a list containing the certain neighborhoods
manhattan_pop["NTA Name"]= manhattan_pop["NTA Name"].str.split("-")
manhattan_pop.head()

,NTA Name,Population
0,"[Marble Hill, Inwood]",46746
1,"[Central Harlem North, Polo Grounds]",75282
2,[Hamilton Heights],48520
3,[Manhattanville],22950
4,[Morningside Heights],55929


In [131]:
#add length of each neighborhood-list as new column
manhattan_pop["len"]= manhattan_pop["NTA Name"].apply(lambda x: len(x))

In [132]:
#iterate over dataframe and append all neighborhoods as single to a new list
manhattan_pop_clean= pd.DataFrame(columns=["NTA Name","Population","len"])

for index, row in manhattan_pop.iterrows():
    if manhattan_pop.iloc[index]["len"] > 1:
        l= manhattan_pop.iloc[index]["len"]
        for n in range(l):
            #print(manhattan_pop.iloc[index]["NTA Name"][n])
            #print(manhattan_pop.iloc[index]["Population"]/ l)
            manhattan_pop_clean= manhattan_pop_clean.append({'NTA Name' : manhattan_pop.iloc[index]["NTA Name"][n],
                                      'Population' : manhattan_pop.iloc[index]["Population"]/ l,
                                      'len' : 1} , 
                                      ignore_index=True)
    else:
        manhattan_pop_clean= manhattan_pop_clean.append({'NTA Name' : manhattan_pop.iloc[index]["NTA Name"][0],
                                      'Population' : manhattan_pop.iloc[index]["Population"],
                                      'len' : 1} , 
                                      ignore_index=True)

manhattan_pop_clean

,NTA Name,Population,len
0,Marble Hill,23373.00,1
1,Inwood,23373.00,1
2,Central Harlem North,37641.00,1
3,Polo Grounds,37641.00,1
4,Hamilton Heights,48520.00,1
5,Manhattanville,22950.00,1
6,Morningside Heights,55929.00,1
7,Central Harlem South,43383.00,1
8,Upper West Side,132378.00,1
9,Hudson Yards,17537.50,1


In [133]:
#drop last 4 rows as they are no real neighborhoods
manhattan_pop_clean.drop([43,44,45,46], inplace= True)
manhattan_pop_clean.drop(["len"], axis=1, inplace= True)
manhattan_pop_clean.head()

,NTA Name,Population
0,Marble Hill,23373.0
1,Inwood,23373.0
2,Central Harlem North,37641.0
3,Polo Grounds,37641.0
4,Hamilton Heights,48520.0


In [244]:
#merge the population data with the manhattan data
manhattan_merge= manhattan_data

manhattan_merge= manhattan_merge.merge(manhattan_pop_clean, left_on="Neighborhood", right_on="NTA Name")
manhattan_merge.drop(["Borough","NTA Name"], axis= 1, inplace= True)
manhattan_merge.head()

,Neighborhood,Latitude,Longitude,Population
0,Marble Hill,40.876551,-73.910660,23373.0
1,Chinatown,40.715618,-73.994279,47844.0
2,Inwood,40.867684,-73.921210,23373.0
3,Hamilton Heights,40.823604,-73.949688,48520.0
4,Manhattanville,40.816934,-73.957385,22950.0


Lastly we add a relative population score to each neighborhood, as this is needed for further analysis. This Population Score is simply the relative Manhattan population.

In [142]:
manhattan_population= manhattan_merge["Population"].sum()

manhattan_merge["Pop Score"]= (manhattan_merge["Population"]/manhattan_population) * 100
manhattan_merge.sort_values(["Pop Score"], ascending=False).head(10)

,Neighborhood,Latitude,Longitude,Population,Pop Score
9,Upper West Side,40.787658,-73.977059,132378.0,12.487813
6,Yorkville,40.775930,-73.947118,77942.0,7.352620
16,Lower East Side,40.717807,-73.980890,72957.0,6.882363
18,West Village,40.734434,-74.006180,66880.0,6.309092
10,Lincoln Square,40.773529,-73.985338,61489.0,5.800534
19,Morningside Heights,40.808000,-73.963896,55929.0,5.276035
3,Hamilton Heights,40.823604,-73.949688,48520.0,4.577110
1,Chinatown,40.715618,-73.994279,47844.0,4.513340
11,Clinton,40.759101,-73.996119,45884.0,4.328444
15,East Village,40.727847,-73.982226,44136.0,4.163548


## Pharmacy Data

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and find data about pharmacys within the given neighborhoods.

In [145]:
#CLIENT_ID = # your Foursquare ID
#CLIENT_SECRET =  # your Foursquare Secret
#VERSION = '20180605' # Foursquare API version

print('Credentials have been removed for privacy issues')

Credentials have been removed for privacy issues


Procedure for requesting and transforming venue data:
1) create a dataframe that request the top100 venues within radius of 500 for each neighborhood.
2) perform one hot encoding for all venue categories
3) group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
4) ommit every other category than pharmacy and calculate a pharmacy score

In [217]:
#create a function to repeat the api request same process to all the neighborhoods in Manhattan
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    category_id= "4bf58dd8d48988d10f951735" #category code for pharmacy (from API docu)
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryID={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT,
            category_id)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [218]:
#apply function to all neighborhoods
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )

Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


In [219]:
#perform one hot encoding for all venue categories
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 

# get a list of columns
cols = list(manhattan_onehot)
# move the column to head of list using index, pop and insert
cols.insert(0, cols.pop(cols.index('Neighborhood')))
cols

# use ix to reorder
manhattan_onehot = manhattan_onehot.ix[:, cols]

manhattan_onehot.head()

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Bar,Beer Garden,Beer Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Bookstore,College Cafeteria,Comedy Club,Community Center,Concert Hall,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Financial or Legal Service,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music School,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Outdoors & Recreation,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Physical Therapist,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Public Art,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,South American Restaur

In [220]:
#group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Bar,Beer Garden,Beer Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Bookstore,College Cafeteria,Comedy Club,Community Center,Concert Hall,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Financial or Legal Service,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music School,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Outdoors & Recreation,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Physical Therapist,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Public Art,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,South American Restaur

In [242]:
#ommit every other category than pharmacy and calculate a pharmacy score
manhattan_pharmacies= manhattan_grouped[["Neighborhood","Pharmacy"]]
manhattan_pharmacies["Pharmacy Score"]= manhattan_pharmacies["Pharmacy"] * 100
manhattan_pharmacies

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Neighborhood,Pharmacy,Pharmacy Score
0,Battery Park City,0.000000,0.000000
1,Carnegie Hill,0.000000,0.000000
2,Central Harlem,0.000000,0.000000
3,Chelsea,0.000000,0.000000
4,Chinatown,0.010000,1.000000
5,Civic Center,0.000000,0.000000
6,Clinton,0.000000,0.000000
7,East Harlem,0.025641,2.564103
8,East Village,0.000000,0.000000
9,Financial District,0.000000,0.000000


## Merge and evaluate data 

In [241]:
#merge pharmacy score to manhattan_merge
manhattan_fin= manhattan_merge.merge(manhattan_pharmacies[["Pharmacy Score","Neighborhood"]], 
                                     left_on="Neighborhood", 
                                     right_on="Neighborhood")
manhattan_fin

,Neighborhood,Latitude,Longitude,Population,Pop Score,Pharmacy Score
0,Marble Hill,40.876551,-73.910660,23373.0,2.204880,4.000000
1,Chinatown,40.715618,-73.994279,47844.0,4.513340,1.000000
2,Inwood,40.867684,-73.921210,23373.0,2.204880,3.448276
3,Hamilton Heights,40.823604,-73.949688,48520.0,4.577110,0.000000
4,Manhattanville,40.816934,-73.957385,22950.0,2.164977,0.000000
5,Upper East Side,40.775639,-73.960508,30603.5,2.886966,0.000000
6,Yorkville,40.775930,-73.947118,77942.0,7.352620,1.000000
7,Lenox Hill,40.768113,-73.958860,40385.5,3.809746,0.000000
8,Roosevelt Island,40.762160,-73.949168,40385.5,3.809746,0.000000
9,Upper West Side,40.787658,-73.977059,132378.0,12.487813,0.000000


1) Consider both population and pharmacy score for final evaluation. They should be weigthed equal as both are important factors for health care system and drug demand.

2) Calculate a weighted final score.

3) Sort neighborhood by this score in order to find the best neighborhood for a pharmacy.

In [267]:
#define weights for both factors
pharmacy_weigth= 0.5
population_weigth= 0.5

#calculate overall score
manhattan_fin["Overall Score"]= manhattan_fin["Pop Score"] * population_weigth - \
                                manhattan_fin["Pharmacy Score"] * pharmacy_weigth
    
#sort values
manhattan_fin.sort_values(["Overall Score"], ascending= False, inplace= True)

#display
manhattan_fin

,Neighborhood,Latitude,Longitude,Population,Pop Score,Pharmacy Score,Overall Score
6,Upper West Side,40.787658,-73.977059,132378.0,12.487813,0.000000,6.243907
4,Yorkville,40.775930,-73.947118,77942.0,7.352620,1.000000,3.176310
16,West Village,40.734434,-74.006180,66880.0,6.309092,0.000000,3.154546
15,Lincoln Square,40.773529,-73.985338,61489.0,5.800534,1.041667,2.379434
23,Hamilton Heights,40.823604,-73.949688,48520.0,4.577110,0.000000,2.288555
11,Clinton,40.759101,-73.996119,45884.0,4.328444,0.000000,2.164222
5,East Village,40.727847,-73.982226,44136.0,4.163548,0.000000,2.081774
25,Roosevelt Island,40.762160,-73.949168,40385.5,3.809746,0.000000,1.904873
19,Lenox Hill,40.768113,-73.958860,40385.5,3.809746,0.000000,1.904873
0,Chinatown,40.715618,-73.994279,47844.0,4.513340,1.000000,1.756670


## Perform Cluster Analysis

In [265]:
# set number of clusters
kclusters = 5

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# generate dataframe with the clusters for each neighborhood
manhattan_cluster= pd.DataFrame([manhattan_grouped["Neighborhood"],kmeans.labels_]).T
manhattan_cluster.rename(columns={"Unnamed 0": "Cluster"}, inplace= True)
manhattan_cluster.head()

,Neighborhood,Cluster
0,Battery Park City,1
1,Carnegie Hill,2
2,Central Harlem,2
3,Chelsea,2
4,Chinatown,0


In [266]:
#merge to manhattan_fin
manhattan_fin_merge= manhattan_fin

manhattan_fin_merge= manhattan_fin_merge.merge(manhattan_cluster, on="Neighborhood", how="left")
manhattan_fin_merge.sort_values("Cluster", inplace=True)
manhattan_fin_merge

,Neighborhood,Latitude,Longitude,Population,Pop Score,Pharmacy Score,Overall Score,Cluster
27,Upper West Side,40.787658,-73.977059,132378.0,12.487813,0.000000,6.243907,0
21,East Village,40.727847,-73.982226,44136.0,4.163548,0.000000,2.081774,0
18,Chinatown,40.715618,-73.994279,47844.0,4.513340,1.000000,1.756670,0
17,Lower East Side,40.717807,-73.980890,72957.0,6.882363,3.921569,1.480397,0
26,Yorkville,40.775930,-73.947118,77942.0,7.352620,1.000000,3.176310,0
9,Gramercy,40.737210,-73.981376,27988.0,2.640234,1.234568,0.702833,0
7,Midtown South,40.748510,-73.988713,14315.0,1.350398,0.000000,0.675199,0
12,Battery Park City,40.711932,-74.016869,19849.5,1.872493,0.000000,0.936246,1
0,Marble Hill,40.876551,-73.910660,23373.0,2.204880,4.000000,-0.897560,1
2,Midtown,40.754691,-73.981669,14315.0,1.350398,1.000000,0.175199,1
